In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from PIL import Image
from matplotlib import rc

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

#import EDA

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

#nlp
# from bs4 import BeautifulSoup
# from urllib.request import urlopen
# import requests
# from selenium import webdriver
# from tqdm.notebook import tqdm
# import time, urllib
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# keys = Keys()
# import re, os, sys, json

# from hanspell import spell_checker

# from konlpy.tag import Kkma #형태소분석
# from konlpy.tag import Okt #형태소분석
# import soynlp; from soynlp.normalizer import * #정규화
# # from pykospacing import Spacing------------------>써야함!
# # spacing = Spacing()------------------------------>써야함!

# from konlpy.tag import Komoran, Hannanum, Kkma, Okt
# komoran = Komoran(); hannanum = Hannanum(); kkma = Kkma(); okt = Okt();

In [6]:
df = pd.read_pickle('data/real_final_df_230115.pickle')
df =df.astype({'shoulder_big':'int','shoulder_small':'int',
               'chongjang_big':'int','chongjang_small':'int',
               'total_big':'int','total_small':'int',
               'arm_big':'int','arm_small':'int',
               'chest_big':'int','chest_small':'int',})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19853 entries, 0 to 20036
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user               19853 non-null  object 
 1   gender             19853 non-null  object 
 2   height             19853 non-null  float64
 3   weight             19853 non-null  float64
 4   item               19853 non-null  object 
 5   size               19853 non-null  object 
 6   star               19853 non-null  float64
 7   content            19853 non-null  object 
 8   size_eval          19853 non-null  object 
 9   bright_eval        19853 non-null  object 
 10  color_eval         19853 non-null  object 
 11  thick_eval         19853 non-null  object 
 12  cm                 19853 non-null  object 
 13  총장                 19853 non-null  float64
 14  어깨너비               19035 non-null  float64
 15  가슴단면               19853 non-null  float64
 16  소매길이               198

In [11]:
df['total_big'].head()

0    1
1    0
2    0
3    0
4    0
Name: total_big, dtype: int32

In [28]:

    
df_total_big        = df[df['total_big'] == 1]
df_total_small      = df[df['total_small'] == 1]
df_total_soso       = df[(df['total_big'] ==0) & (df['total_small'] ==0)]

df_chongjang_big    = df[df['chongjang_big'] == 1]
df_chongjang_small  = df[df['chongjang_small'] == 1]
df_chongjang_soso   = df[(df['chongjang_big'] == 0) & (df['chongjang_small'] == 0)]

df_shoulder_big     = df[df['shoulder_big'] == 1]
df_shoulder_small   = df[df['shoulder_small'] == 1]
df_shoulder_soso    = df[(df['shoulder_big'] == 0) & (df['shoulder_small'] == 0)]

df_chest_big        = df[df['chest_big'] == 1]
df_chest_small      = df[df['chest_small'] == 1]
df_chest_soso       = df[(df['chest_big'] == 0 )& (df['chest_small'] == 0)]

df_arm_big          = df[df['arm_big'] == 1]
df_arm_small        = df[df['arm_small'] == 1]
df_arm_soso         = df[(df['arm_big'] == 0) & (df['arm_small'] == 0)]

###
df_total_big        = df_total_big[['height','weight','총장','어깨너비','가슴단면','소매길이']]
df_total_small      = df_total_small[['height','weight','총장','어깨너비','가슴단면','소매길이']]
df_total_soso       = df_total_soso[['height','weight','총장','어깨너비','가슴단면','소매길이']]

df_chongjang_big    = df_chongjang_big[['height','weight','총장']]
df_chongjang_small  = df_chongjang_small[['height','weight','총장']]
df_chongjang_soso   = df_chongjang_soso[['height','weight','총장']]

df_shoulder_big     = df_shoulder_big[['height','weight','어깨너비']]
df_shoulder_small   = df_shoulder_small[['height','weight','어깨너비']]
df_shoulder_soso    = df_shoulder_soso[['height','weight','어깨너비']]

df_chest_big        = df_chest_big[['height','weight','가슴단면']]
df_chest_small      = df_chest_small[['height','weight','가슴단면']]
df_chest_soso       = df_chest_soso[['height','weight','가슴단면']]

df_arm_big          = df_arm_big[['height','weight','소매길이']]
df_arm_small        = df_arm_small[['height','weight','소매길이']]
df_arm_soso         = df_arm_soso[['height','weight','소매길이']]


print('---크다. 작다. 보통---')
print('전체{},{},{}'.format(df_total_big.shape,df_total_small.shape,df_total_soso.shape))
print('총장{},{},{}'.format(df_chongjang_big.shape,df_chongjang_small.shape,df_chongjang_soso.shape))
print('어깨{},{},{}'.format(df_shoulder_big.shape,df_shoulder_small.shape,df_shoulder_soso.shape))
print('가슴{},{},{}'.format(df_chest_big.shape,df_chest_small.shape,df_chest_soso.shape))
print('팔{},{},{}'.format(df_arm_big.shape,df_arm_small.shape,df_arm_soso.shape))

df_total_big.dropna(axis = 0,inplace = True)
df_total_small.dropna(axis = 0,inplace = True)
df_total_soso.dropna(axis = 0,inplace = True)

df_chongjang_big.dropna(axis = 0,inplace = True)
df_chongjang_small.dropna(axis = 0,inplace = True)
df_chongjang_soso.dropna(axis = 0,inplace = True)

df_arm_big.dropna(axis = 0,inplace = True)
df_arm_small.dropna(axis = 0,inplace = True)
df_arm_soso.dropna(axis = 0,inplace = True)

df_chest_big.dropna(axis = 0,inplace = True)
df_chest_small.dropna(axis = 0,inplace = True)
df_chest_soso.dropna(axis = 0,inplace = True)

df_shoulder_big.dropna(axis = 0,inplace = True)
df_shoulder_small.dropna(axis = 0,inplace = True)
df_shoulder_soso.dropna(axis = 0,inplace = True)
print('---크다. 작다. 보통---')
print('전체{},{},{}'.format(df_total_big.shape,df_total_small.shape,df_total_soso.shape))
print('총장{},{},{}'.format(df_chongjang_big.shape,df_chongjang_small.shape,df_chongjang_soso.shape))
print('어깨{},{},{}'.format(df_shoulder_big.shape,df_shoulder_small.shape,df_shoulder_soso.shape))
print('가슴{},{},{}'.format(df_chest_big.shape,df_chest_small.shape,df_chest_soso.shape))
print('팔{},{},{}'.format(df_arm_big.shape,df_arm_small.shape,df_arm_soso.shape))


#리스트에 들어가는 순서는 big,small,soso 순!! 꼭지키기!!
hood_chongjang_train_lst = [
                 #df_total_big,df_total_small,df_total_soso,
                  df_chongjang_big,df_chongjang_small,df_chongjang_soso,
                  #df_shoulder_big,df_shoulder_small,df_shoulder_soso,
                  #df_chest_big,df_chest_small,df_chest_soso,
                  #df_arm_big,df_arm_small,df_arm_soso
                  ]
hood_shoulder_train_lst = [
                 #df_total_big,df_total_small,df_total_soso,
                  #df_chongjang_big,df_chongjang_small,df_chongjang_soso,
                  df_shoulder_big,df_shoulder_small,df_shoulder_soso,
                  #df_chest_big,df_chest_small,df_chest_soso,
                  #df_arm_big,df_arm_small,df_arm_soso
                  ]

hood_chest_train_lst = [
                 #df_total_big,df_total_small,df_total_soso,
                  #df_chongjang_big,df_chongjang_small,df_chongjang_soso,
                  #df_shoulder_big,df_shoulder_small,df_shoulder_soso,
                  df_chest_big,df_chest_small,df_chest_soso,
                  #df_arm_big,df_arm_small,df_arm_soso
                  ]

hood_arm_train_lst = [
                 #df_total_big,df_total_small,df_total_soso,
                  #df_chongjang_big,df_chongjang_small,df_chongjang_soso,
                  #df_shoulder_big,df_shoulder_small,df_shoulder_soso,
                  #df_chest_big,df_chest_small,df_chest_soso,
                  df_arm_big,df_arm_small,df_arm_soso
                  ]
#전체적으로 크다 혹은 작다 라고 답한 사람들은 그냥 총장, 어깨, 가슴, 팔에 전부 1을 주는 것이 어떤지,,
#아니면 전체모델을 따로 돌리던가,, 하 모르겄다

---크다. 작다. 보통---
전체(3942, 6),(248, 6),(15663, 6)
총장(657, 3),(331, 3),(18865, 3)
어깨(200, 3),(34, 3),(19619, 3)
가슴(115, 3),(25, 3),(19713, 3)
팔(458, 3),(28, 3),(19367, 3)
---크다. 작다. 보통---
전체(3798, 6),(237, 6),(15000, 6)
총장(657, 3),(331, 3),(18865, 3)
어깨(191, 3),(32, 3),(18812, 3)
가슴(115, 3),(25, 3),(19713, 3)
팔(458, 3),(28, 3),(19366, 3)


## 훈련

In [38]:
import joblib
def trainModel(lst,sizetype):
    user_pred_list = []
    for i in range(len(lst)):
        X = lst[i].iloc[:, :-1]
        y = lst[i].iloc[:, -1]

        xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 531)
        
        mseOos = []
        nTreeList = range(50, 500, 10)

        for iTrees in tqdm(nTreeList, desc='iterate list'):
            depth = None
            RFModel = RandomForestRegressor(n_estimators=iTrees,
                                            max_depth=depth,
                                            oob_score=False, 
                                            random_state=531)
            RFModel.fit(xTrain, yTrain)
            
            #데이터 세트에 대한 MSE 누적
            prediction = RFModel.predict(xTest)
            mseOos.append(mean_squared_error(yTest, prediction))
            
        regr = RandomForestRegressor(n_estimators = nTreeList[np.argmin(mseOos)],
                                random_state=531)
        regr.fit(xTrain, yTrain)
        
        prediction = regr.predict(xTest)
        if i % 3 == 1:
            print("MSE of small {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(regr, './'+sizetype+'_small_RFModel.pkl')
        elif i % 3 == 2:
            print("MSE of soso {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(regr, './'+sizetype+'_soso_RFModel.pkl')
        elif i % 3 == 0:
            print("MSE of big {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(regr, './'+sizetype+'_big_RFModel.pkl')
            
        userTest = [[178, 76]]
        prediction_user = regr.predict(userTest)
        user_pred_list.append(prediction_user)
        print("User Test of {}'s Model : {}".format(lst[i].columns[-1], prediction_user))
    

In [39]:
sizetype = 'chongjang'
trainModel(hood_chongjang_train_lst,sizetype)

iterate list: 100%|██████████| 45/45 [00:24<00:00,  1.87it/s]


MSE of big 총장's Model : 18.818416058042246
User Test of 총장's Model : [71.31904762]


iterate list: 100%|██████████| 45/45 [00:19<00:00,  2.26it/s]


MSE of small 총장's Model : 71.30255563849123
User Test of 총장's Model : [69.684]


iterate list: 100%|██████████| 45/45 [02:07<00:00,  2.84s/it]


MSE of soso 총장's Model : 13.186106683936623
User Test of 총장's Model : [71.26602641]


In [ ]:
sizetype = 'chest'
trainModel(hood_chest_train_lst,sizetype)
sizetype = 'arm'
trainModel(hood_arm_train_lst,sizetype)
sizetype = 'shoulder'
trainModel(hood_shoulder_train_lst,sizetype)

## 비고

In [ ]:
import pycaret.regression

In [ ]:
## pycaret
demo = pycaret.regression.setup(data = hood_train_lst[0], target = 'outseam', 
                                # ignore_features = [],
                                # normalize = True,
                                # transformation= True,
                                # transformation_method = 'yeo-johnson',
                                # transform_target = True,
                                # remove_outliers= True,
                                # remove_multicollinearity = True,
                                # ignore_low_variance = True,
                                # combine_rare_levels = True
                                ) 

best = pycaret.regression.compare_models()
# plot_model(best)
# evaluate_model(best)

## Creating models for the best estimators
random_forest = pycaret.regression.create_model('rf')

# ## Tuning the created models 
# random_forest = pycaret.tune_model(random_forest)

## Finaliszing model for predictions 
test_data = data_outseam.iloc[:10, :]
predictions = pycaret.regression.predict_model(random_forest, data = test_data)